In [1]:
import pandas as pd
from pickle import load

In [2]:
df = load(open('song_list_v7.pkl', 'rb'))

### Re-labeling bins to standardize include,exclude

In [3]:
# shift end-year-label to follow pattern of exlcuding end-year
df['by_decade_and_quinquennial'] = df['by_decade_and_quinquennial'].replace('2015-2019','2015-2020')

# where year is 1950 and quin bin is >50, replace quin bin with 50-60
df.loc[df['year'].eq(1950) & df['by_decade_and_quinquennial'].eq('>50s'), 'by_decade_and_quinquennial'] = '50-60'

# resetting index to remove missing indexes
df = df.reset_index(drop=True)

# rename for convenience
df = df.rename(columns={'by_decade_and_quinquennial':'quin_bins'})

# removing unwanted columns
df = df.drop(['index'], axis=1);

### Quick custom test to ensure year column value is in correct labeled range

In [4]:
for idx in range(len(df)):
    
    try:
        year = df['year'][idx]
    except KeyError: # Catch missing indexes, should be fixed
        print('.')
        continue
    
    
    # using the bin values from column:'quin_bins', hard set for 2020 and except=(>50)
    try:
        if year == 2020:
            bin_range = [2020, 2021]
        else:
            bin_range = [int(ele) for ele in df['quin_bins'][idx].split('-')]
    except:
        bin_range = [0, 1950]
        
    
    # adding century and millenia to decade abreviations
    for idx, ele in enumerate(bin_range):
        if len(str(ele)) == 2:
            bin_range[idx] = ele + 1900
            
    
    try:
        assert year in range(bin_range[0], bin_range[1])
    except:
        print(idx, year, bin_range)
        break

#### If nothing returns here ^^ , test passed. DISCLAIMER- not a comprehensive test

#### Pickle off new trimmed and binned df

In [ ]:
df.to_pickle('song_list_v7_binned.pkl')